In [1]:
bbox = [36.0, 56.45, 36.1, 56.5]
addr = "\"https://api.openstreetmap.org/api/0.6/map?bbox=" \
    + str(bbox[0])  + ',' + str(bbox[1]) + ',' + str(bbox[2]) + ',' + str(bbox[3]) + '"'
!curl -o request_map.osm $addr

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1925k    0 1925k    0     0  1925k      0 --:--:--  0:00:01 --:--:-- 1599k
100 2966k    0 2966k    0     0  2966k      0 --:--:--  0:00:01 --:--:-- 1977k


___

In [1]:
import xml.etree.ElementTree as ET

In [2]:
root = ET.parse('request_map.osm').getroot()

In [3]:
from pandas import DataFrame

In [63]:
%%time
points = DataFrame(columns=['geometry'])
for node in root.iter('node'):
    node_info = node.attrib
    # node_id = node_info['id']
    node_coords = (float(node_info['lat']), float(node_info['lon']))
    points.loc[node_info['id']] = [node_coords]  # , root.findall('way//nd[@ref="' + node_id + '"]/..')

Wall time: 1min 4s


In [5]:
%%time
points = DataFrame(columns=['geometry', 'ways'])
for node in root.iter('node'):
    node_info = node.attrib
    node_id = node_info['id']
    node_coords = (float(node_info['lat']), float(node_info['lon']))
    points.loc[node_id] = node_coords , root.findall('way//nd[@ref="' + node_id + '"]/..')

Wall time: 9min 34s


In [29]:
ways = DataFrame(columns=['geometry'])
for way in root.iter('way'):
    # way_id = way.attrib['id']
    ways.loc[way.attrib['id']] = [[node.attrib['ref'] for node in way.findall('nd')]]  # , root.findall('relation//member[@ref="' + way_id + '"]/..')

___

In [4]:
# find node with id and return its coords
def node_coords(ID):
    node_info = root.find('node[@id="' + ID + '"]').attrib
    return (float(node_info['lat']), float(node_info['lon']))

In [5]:
# return tuple of coords of all nodes in a way
def way_coords(way):
    coords = list()
    for node in way.findall('nd'):
        coords.append(node_coords(node.attrib['ref']))
    return tuple(coords)

In [6]:
# extract all ways
import pandas as pd
def process_way():
    ways_df = pd.DataFrame(columns=['geometry', 'tag', 'type', 'multi'])
    keys = ('barrier', 'building', 'geological', 'military', 'natural', 'highway', 'landuse', 'railway', 'waterway', 'water')
    for key in keys:
        ways = root.findall('way/tag[@k="' + key + '"]/..')
        for way in ways:
            ways_df.loc[way.attrib['id']] = (way_coords(way), key, way.find('tag[@k="' + key + '"]').attrib['v'], False)
    return ways_df

In [7]:
%%time
ways_df = process_way()

Wall time: 32.1 s


In [68]:
ways_df.head()

,geometry,tag,type,multi
56700280,"((56.4330779, 35.9870501), (56.4292344, 35.984...",natural,wetland,False
98992684,"((56.4835611, 36.0208295), (56.4831855, 36.021...",natural,water,False
491945893,"((56.4912754, 36.0028195), (56.4911155, 36.002...",natural,wood,False
492328723,"((56.4864893, 36.0584807), (56.4864063, 36.057...",natural,wood,False
492328724,"((56.4774604, 36.0253608), (56.4768916, 36.025...",natural,wood,False


___

In [8]:
# find way with id
def find_way(ID):
    way = root.find('way[@id="' + ID + '"]')
    return way_coords(way)

In [9]:
from math import isclose
def is_polygon(ID):
    obj = ways_df[ID]
    p1, p2 = obj[0], obj[-1]
    return isclose(p1[0], p2[0]) and isclose(p1[1], p2[1])

In [19]:
tags = ('natural', 'landuse')
for tag in tags:
    relations = root.findall('relation/tag[@k="' + tag + '"]/..')
    for relation in relations:
        outer = relation.findall('member[@role="outer"]')
        outer = [way.attrib['ref'] for way in outer]
        if len(outer) == 1:
            ways_df[outer[0].attrib['ref']].multi = True
        else:
            outer = [way for]
            for member in outer:
                if not is_polygon(member.attrib['ref']):
                    ways_df.drop(member.attrib['ref'])

[<Element 'member' at 0x000002662E305540>, <Element 'member' at 0x000002662E305590>, <Element 'member' at 0x000002662E3055E0>]
[<Element 'member' at 0x000002662E340770>]
[<Element 'member' at 0x000002662E340EA0>]
[<Element 'member' at 0x000002662E34A180>]
[<Element 'member' at 0x000002662E34A3B0>]
[<Element 'member' at 0x000002662E340130>]
[<Element 'member' at 0x000002662E340450>]


In [2]:
import os
os.remove("request_map.osm")